In [312]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

In [ ]:
if (date.today()- pd.DateOffset(days=1)).weekday() == 6:
    today = (date.today()- pd.DateOffset(days=3)).strftime('%Y-%m-%d')
else:
    today = (date.today()- pd.DateOffset(days=1)).strftime('%Y-%m-%d')
print(today)

start_date = date.today() - pd.DateOffset(days=8)
start_date = start_date.strftime('%Y-%m-%d')
print(start_date)
#start_date  ='2024-10-01'

In [ ]:
date_range = pd.date_range(start=start_date, end=today)

for single_date in date_range:
    print(single_date.strftime('%Y-%m-%d'))

In [315]:
# today = '2024-04-25'

In [316]:
reason_codes = [
    "Hub SB",
    "WWSB",
    "Broker buy",
    "Return from ODM",
    "To be re-programmed",
    "To be sorted",
    "To be qualified",
    "FXN internal SB",
    "Append EM",
    "Navy deal",
    "EOL without any supply",
    "Waiver process",
    "Others"]

### get external data

In [ ]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
print('connect')

# download shortge from SQL


cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate >= '{start_date}' and ReportDate <= '{today}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
shortage_ext['HP_PN'] = shortage_ext['HP_PN'].str.replace('\n', ',', regex=False)
shortage_ext['FV'] = shortage_ext['FV'].str.replace('\n', ',', regex=False)

# download PNFV from SQL
cursor.execute("SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV = PNFV.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
PNFV = PNFV.rename({'Descr':'FV'})

In [ ]:
PNFV

In [ ]:
shortage_ext['ReportDate'] = pd.to_datetime(shortage_ext['ReportDate'])

# group bu commodity and count the len of each group
shortage_ext = shortage_ext[shortage_ext['P1'] != 0]

# CPU need to add supplier
ext_CPU = shortage_ext[shortage_ext['Commodity'] == 'CPU']
ext_CPU['Supplier'] = np.where(ext_CPU['FV'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
ext_CPU_group = ext_CPU.groupby(['ReportDate','Commodity','Supplier']).size().reset_index(name='Count')
ext_CPU_group['Commodity'] = ext_CPU_group['Supplier'] + ' ' +  ext_CPU_group['Commodity']
ext_CPU_group.drop(columns=['Supplier'], inplace=True)

### get internal data

In [ ]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview WHERE [Report Date] >= '{start_date}' AND [Report Date] <= '{today}'")
internal = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# check if there is empty value in Commodity
if internal.Commodity.isna().any():
    print(internal[internal.Commodity.isna()])
else:
    print('No empty value in Commodity')

In [ ]:
# drop the rows with 'Delete' == 'Y' and group by Commodity
internal = internal[(~(internal['Delete'] == 'Y'))]

#drop internal P1 = 0 and na
internal = internal.dropna(subset=['P1'])
internal = internal[internal['P1'] != 0]

internal.rename(columns={'Report Date': 'ReportDate'}, inplace=True)
internal['ReportDate'] = pd.to_datetime(internal['ReportDate'])

# CPU need to add supplier
int_CPU = internal[internal['Commodity'] == 'CPU']
int_CPU['Supplier'] = np.where(int_CPU['FV Des'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
int_CPU_group = int_CPU.groupby(['ReportDate','Commodity','Supplier']).size().reset_index(name='Count')
int_CPU_group['Commodity'] = int_CPU_group['Supplier'] + ' ' +  int_CPU_group['Commodity']
int_CPU_group.drop(columns=['Supplier'], inplace=True)

Special supply format misaligned (in reason_codes)

In [322]:
df_special_supply = internal.copy()
#special supply is non supplier supply
df_special_supply['ETA'] = df_special_supply['ETA'].str.split('\n')
df_special_supply = df_special_supply.explode('ETA')
text = df_special_supply['ETA']

In [ ]:
def check_conditions(text):
   # 检查文本是否以 'ETA' 或 'TBD' 开头
   if text is None:
        return False
   
   if text.startswith('ETA') or text.startswith('TBD'):
       return False
   # 检查文本是否包含任何 reason_codes 中的字符串
   for reason in reason_codes:
       if reason in text:
           return True
   return False
# 应用函数到 'ETA' 列
df_special_supply['ETA misaligned'] = df_special_supply['ETA'].apply(check_conditions)
# 打印结果
print(df_special_supply)
df_spsupply_true = df_special_supply[df_special_supply['ETA misaligned'] == True]
df_spsupply_mis = df_spsupply_true[['ReportDate', 'Buyer Name', 'Commodity', 'FV Des', 'ETA']]

In [ ]:
shortage_ext.head()

In [ ]:
df_spsupply_mis.head()

In [ ]:
CQODM = ['CQQCI', 'CQIEC', 'CQWIS', 'TWIEC', 'L_WHFXN', 'CQPCQ', 'MSI', 'PCQ', 'PCP', 'WHFXN_L5', 'PSZ', 'FLH', 'TWQCI', 'SZBYD']
WHODM = ['WHFXN']
KSODM = ['CEI']
THODM = ['THQCI','THIEC']
MXODM = ['IMX']

ODM_list = CQODM + WHODM + KSODM + THODM + MXODM

incorrect_ODM = []
for index in internal.index:
    if internal.loc[index,'ODM'] not in ODM_list:
        incorrect_ODM.append(internal.loc[index,:])
    else:
        pass

incorrect_ODM = pd.DataFrame(incorrect_ODM)
try:
    incorrect_ODM = incorrect_ODM[['Report Date', 'Buyer Name','Commodity','ODM']]
except:
    pass

incorrect_ODM = incorrect_ODM.drop_duplicates()



print(len(incorrect_ODM))

In [ ]:
Commodity_list = ['AC ADAPTOR','AudioAMP','AudioCodec','BATTERY','CardReader','CHIPSET','CONNECTOR','CPU','EC','eMMC', 'Ethernet IC','GPU','GPU-Graphic Card','HDD','LanChip','LCD','Memory','NIC',
 'ODD','POWERCORDS','PSU','Retimer','SIO','SSS','ThunderBT','TPM','TS','USBIC','VRAM','WEBCAM','WLAN','WWAN']

incorrect_commodity = []
for index in internal.index:
    if internal.loc[index,'Commodity'] not in Commodity_list:
        incorrect_commodity.append(internal.loc[index,:])
    else:
        pass

incorrect_commodity = pd.DataFrame(incorrect_commodity)
print(len(incorrect_commodity))

### (archive) merge and compare internal with external

In [330]:
# int_counts = internal.groupby(['ReportDate', 'Commodity']).size().reset_index(name='Count')
# ext_count = shortage_ext.groupby(['ReportDate', 'Commodity']).size().reset_index(name='Count')
# df_compare = int_counts[(int_counts['Commodity'] != 'CPU')].merge(ext_count[(ext_count['Commodity'] != 'CPU')],on=['ReportDate', 'Commodity'],how='outer',suffixes=('_int', '_ext'))
# df_compare_CPU = int_CPU_group.merge(ext_CPU_group,on=['ReportDate','Commodity'],how='outer',suffixes=('_int', '_ext'))
# df_compare = pd.concat([df_compare, df_compare_CPU], axis=0)
# df_compare = df_compare.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [331]:
#df_compare

In [332]:
# df_mismatch_total = pd.DataFrame()


# for report_date, group in df_compare.groupby('ReportDate'):
#     df_mismatch = group[((group['Count_int'].isnull()) & (group['Count_ext'].notnull())) | ((group['Count_int'].notnull()) & (group['Count_ext'].isnull()))]
#     df_mismatch.loc[:, ['Count_int', 'Count_ext']] = df_mismatch.loc[:, ['Count_int', 'Count_ext']].fillna(0)
#     df_mismatch.set_index('Commodity', inplace=True)
#     df_mismatch[['Count_int', 'Count_ext']] = df_mismatch[['Count_int', 'Count_ext']].astype(int)
#     df_mismatch.rename(columns={'Count_int': 'Count_internal', 'Count_ext': 'Count_external'}, inplace=True)
#     df_mismatch = df_mismatch[df_mismatch.index != 'Ethernet IC']
#     df_mismatch['ReportDate'] = report_date
#     df_mismatch_total = pd.concat([df_mismatch_total, df_mismatch], axis=0)

# df_mismatch_total.reset_index(inplace=True)
# df_mismatch_total

### misalignment in HP_PN

In [333]:
# internal/external data for HP_PN check
internal_pn = internal[['ReportDate', 'Commodity', 'HP Part Number', 'Buyer Name']].drop_duplicates()
internal_pn = internal_pn.rename(columns={'HP Part Number': 'HP_PN', 'Buyer Name': 'BuyerName'})
external_pn = shortage_ext[['ReportDate', 'Commodity', 'HP_PN', 'BuyerName']].drop_duplicates()


# prcoess HP_PN format
def preprocess_pn(df):
    df['HP_PN'] = df['HP_PN'].astype(str).str.replace(' ', ',', regex=False)
    df['HP_PN'] = df['HP_PN'].str.replace('\n', '', regex=False)
    df = df.explode('HP_PN').reset_index(drop=True)
    df['HP_PN'] = df['HP_PN'].astype(str).str.strip().str[:6]
    df = df[df['HP_PN'].notna() & (df['HP_PN'] != '')]
    return df.drop_duplicates()

internal_pn = preprocess_pn(internal_pn)
external_pn = preprocess_pn(external_pn)

In [334]:
# first-level comparison: use ReportDate+Commodity+HP_PN as key 
df_mismatch_pn = pd.merge(internal_pn, external_pn, on=['ReportDate', 'Commodity', 'HP_PN'], how='outer', indicator=True)
df_mismatch_pn['BuyerName'] = df_mismatch_pn['BuyerName_y'].combine_first(df_mismatch_pn['BuyerName_x'])
df_mismatch_pn_both = df_mismatch_pn[df_mismatch_pn['_merge'] == 'both']


# second-level comparison(for those mismatched ): use ReportDate+Commodity+Buyername as key
to_remove_internal = df_mismatch_pn_both.groupby(['ReportDate', 'Commodity', 'BuyerName_x']).size().reset_index(name='count')
to_remove_internal = to_remove_internal[to_remove_internal['count'] >= 1][['ReportDate', 'Commodity', 'BuyerName_x']]
to_remove_external = df_mismatch_pn_both.groupby(['ReportDate', 'Commodity', 'BuyerName_y']).size().reset_index(name='count')
to_remove_external = to_remove_external[to_remove_external['count'] >= 1][['ReportDate', 'Commodity', 'BuyerName_y']]

df_mismatch_pn = df_mismatch_pn.loc[
    ~df_mismatch_pn[['ReportDate', 'Commodity', 'BuyerName_x']].apply(tuple, axis=1).isin(to_remove_internal.apply(tuple, axis=1)) &
    ~df_mismatch_pn[['ReportDate', 'Commodity', 'BuyerName_y']].apply(tuple, axis=1).isin(to_remove_external.apply(tuple, axis=1))]


# clean the format
df_mismatch_pn = df_mismatch_pn.groupby(['ReportDate', 'Commodity', 'BuyerName', '_merge'], observed=True).agg({'HP_PN': lambda x: ', '.join(x)}).reset_index()
df_mismatch_pn['Missing PN in'] = df_mismatch_pn['_merge'].map({'right_only': 'Internal: ', 'left_only': 'External: '}) + df_mismatch_pn['HP_PN']

df_mismatch_pn = df_mismatch_pn.drop(columns=['HP_PN', '_merge', 'BuyerName'])
df_mismatch_pn = df_mismatch_pn.drop_duplicates()

### create email

In [335]:
To_list = 'gps.taiwan.nb.buy-sell@hp.com'
CC_list = 'spencer.cheng1@hp.com; howie.chang1@hp.com; frederick.shih@hp.com; bspott@hp.com'

mail = win32.Dispatch("Outlook.Application").CreateItem(0)
mail.To = To_list
mail.CC = CC_list
mail.Subject = f'Internal/ External Report Misalignment alert <From Report Date: {start_date} to {today}>'

message = f"""Hi team,<br><br>"""

if len(df_mismatch_pn) > 0 or len(incorrect_ODM) > 0 or len(incorrect_commodity) > 0 or len(df_spsupply_mis) > 0:
    message += f"""The following shows misalignment in external & internal reports, or incorrect data in internal reports from the report date: <b><font color="blue">{start_date}</b></font>, 
    please review and make the necessary amendments.<br><br>"""

    if len(df_mismatch_pn) > 0:
        message += f"""Below data contains <b><font color="blue">misaligned/ missing data in external & internal reports</b></font>, please upload missing values or correct errors.<br><font color="gray">
        <u>Note: If any below PNs have P1 set to 0 in either external or internal report due to the sorting issue, please ignore the alert.</u></font><br>
        {df_mismatch_pn.to_html(escape=False, index=False)}<br><br>"""

    if len(incorrect_ODM) > 0:
        incorrect_ODM['ODM'] = incorrect_ODM['ODM'].apply(lambda name: f'<font color="Red">{name}</font>')
        message += f"""Below data contains <b><font color="blue">incorrect ODM name</b></font>in internal reports, please delete the data and upload again with the correct ODM name.<br>
        {incorrect_ODM.to_html(escape=False, index=False)}<br><br>"""

    if len(incorrect_commodity) > 0:
        incorrect_commodity['Commodity'] = incorrect_commodity['Commodity'].apply(lambda name: f'<font color="Red">{name}</font>')
        message += f"""Below data contains <b><font color="blue">incorrect Commodity name</b></font>, please delete the data and upload again with the correct Commodity name.<br>
        {incorrect_commodity.to_html(escape=False, index=False)}<br><br>"""

    if len(df_spsupply_mis) > 0:
        message += f"""Below data contains <b><font color="blue">non-supplier supply with incorrect format</b></font>, please update the data with internal shortage tool accordingly.<br>
        {df_spsupply_mis.to_html(escape=False, index=False)}<br><br>"""

if len(df_mismatch_pn) == 0 and len(incorrect_ODM) == 0 and len(incorrect_commodity) == 0 and len(df_spsupply_mis) == 0:
    message = f"""Hi team,<br><br>
    No information is <b><font color="blue">misaligned/ missing</b></font> in external & internal reports or incorrect data in internal reports from report date: <b><font color="blue">{start_date} to {today}</b></font>.<br><br>"""

message += """If you have any questions or need assistance, please feel free to log a ticket through the following link: 
<a href="https://forms.office.com/pages/responsepage.aspx?id=ooF5ylp4PUa4Kj24ffw85u5RnrVmIrxMqwWn4XKOaL9UNlRST1VDQUtUTVA5VDhTTzlBU1lGRjdaNy4u&origin=lprLink&route=shorturl">Digital tool issue ticket</a>.<br>
Thank you.<br><br>"""


signature = """<br><br><font color="black">Best Regards,<br>Operations Transformation Team</font>"""

mail.GetInspector
index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body'))
mail.HTMLBody = message + signature
mail.display()


In [ ]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] = '{today}'")
shor = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
conn.close